# Linear Model and Regularization with PySpark

Linear model is one of the most simple machine learning algorithm. People often getting attracted to more advanced model such as Neural Network or Gradient Boosting due to the hype and the preditictive performance. However, on most of daily business case, building a linear model is good enough. Linear model is also comes with the benefit of being interpretable, compared to the black box Neural Network. On this occasion, we will build a linear model with the addition of regularization to analyze the data while still get a great predictive performance.

## Library and Setup

As part of my learning journey, I will use PySpark throughout the notebook. First, we will start the spark session and name it as `laptop_lm`.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('laptop_lm').getOrCreate()

spark

The following code contains all libraries used in this notebook.

In [2]:
# Data Wrangling
import pandas as pd
import numpy as np

# Schema
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

# Spark function
from pyspark.sql import functions as f

# One-Hot Encoding
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

# Scaling
from pyspark.ml.feature import StandardScaler

# Cross-Validation
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

# Machine Learning
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Case Study: Laptop Price Prediction

On this occasion, we will try to understand what makes a price of a laptop to increase by building a linear model. For a computer geek or people who manufactured laptops may already now the production cost of each component. However, for a lay people like us who only know how to use the laptop, exploring this dataset and building a machine learning model around it will help us to compare laptop with various specifications and build by various companies. We may also see some intangible factors that can affect the price, such as the value of a brand like Apple or the CPU component such us Intel Core vs AMD.

## Data

The data come from [Laptop Company Price List](https://www.kaggle.com/muhammetvarl/laptop-price) with the following dictionary:

- **Company**: Laptop Manufacturer
- **Product**: Brand and Model
- **TypeName**: Type (Notebook, Ultrabook, Gaming, etc.)
- **Inches**: Screen Size
- **ScreenResolution**: Screen Resolution
- **Cpu**: Central Processing Unit (CPU)
- **Ram**: Laptop RAM
- **Memory**: Hard Disk / SSD Memory
- **GPU**: Graphics Processing Units (GPU)
- **OpSys**: Operating System
- **Weight**: Laptop Weight
- **Price_euros**: Price in Euro

In [3]:
# Read data
laptop = spark.read.csv("data/laptop_price.csv", header= True, inferSchema=True)

# Show the first 20 rows from the data
laptop.show()

+---------+-------+-----------------+---------+------+--------------------+--------------------+----+-------------------+--------------------+----------+------+-----------+
|laptop_ID|Company|          Product| TypeName|Inches|    ScreenResolution|                 Cpu| Ram|             Memory|                 Gpu|     OpSys|Weight|Price_euros|
+---------+-------+-----------------+---------+------+--------------------+--------------------+----+-------------------+--------------------+----------+------+-----------+
|        1|  Apple|      MacBook Pro|Ultrabook|  13.3|IPS Panel Retina ...|Intel Core i5 2.3GHz| 8GB|          128GB SSD|Intel Iris Plus G...|     macOS|1.37kg|    1339.69|
|        2|  Apple|      Macbook Air|Ultrabook|  13.3|            1440x900|Intel Core i5 1.8GHz| 8GB|128GB Flash Storage|Intel HD Graphics...|     macOS|1.34kg|     898.94|
|        3|     HP|           250 G6| Notebook|  15.6|   Full HD 1920x1080|Intel Core i5 720...| 8GB|          256GB SSD|Intel HD Graph

Check the type of the dataframe.

In [4]:
type(laptop)

pyspark.sql.dataframe.DataFrame

You can also convert the Spark dataframe to Pandas dataframe for better visual.

In [5]:
pd.DataFrame(laptop.head(5), columns= laptop.columns)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


Let's check the schema or the data type of each column.

In [6]:
laptop.printSchema()

root
 |-- laptop_ID: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- TypeName: string (nullable = true)
 |-- Inches: double (nullable = true)
 |-- ScreenResolution: string (nullable = true)
 |-- Cpu: string (nullable = true)
 |-- Ram: string (nullable = true)
 |-- Memory: string (nullable = true)
 |-- Gpu: string (nullable = true)
 |-- OpSys: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- Price_euros: double (nullable = true)



Some column should be numerical, such as the RAM and memory. Those columns is still in string format and have non-numeric characters. We will clean the data later before building the model.

We will continue by checking the number of rows and columns from the data.

In [7]:
# Number of Row
print( 'Number of rows: ' + str( laptop.count() ) )

# Number of Column
print( 'Number of columns: ' + str( len(laptop.columns) ) )

Number of rows: 1303
Number of columns: 13


Let's check if there is any duplicated data. We need to group the data with all columns and then count the number of each group. A duplicate row will have count > 1.

In [8]:
laptop.groupBy(laptop.columns).count().filter("count > 1").show()

+---------+-------+-------+--------+------+----------------+---+---+------+---+-----+------+-----------+-----+
|laptop_ID|Company|Product|TypeName|Inches|ScreenResolution|Cpu|Ram|Memory|Gpu|OpSys|Weight|Price_euros|count|
+---------+-------+-------+--------+------+----------------+---+---+------+---+-----+------+-----------+-----+
+---------+-------+-------+--------+------+----------------+---+---+------+---+-----+------+-----------+-----+



Based on the finding, we will have 0 observation of duplicated data.

Let's check if there is any missing value from each column. We can do this by substracting the number of original row with the number of rows after removing the NA.

In [9]:
original_row = laptop.count()
after_na = laptop.na.drop().count()

print("Number of rows with missing value:")
original_row - after_na

Number of rows with missing value:


0

Based on the result, we find that there is no missing value in any column on our dataset.

## Data Wrangling

Although the information given from the dataset is quite comprehensive, we need to transform the data to a proper format to build a machine learning.

### Transforming Weight

The first we do is removing the weight unit (kg) from the `Weight` column and tranform the value into float/numeric.

In [10]:
laptop = laptop.withColumn('Weight', f.regexp_replace(laptop['Weight'], 'kg', ''))
laptop = laptop.withColumn('Weight', laptop['Weight'].cast(DoubleType()))

laptop.show(10)

+---------+-------+---------------+---------+------+--------------------+--------------------+----+-------------------+--------------------+----------+------+-----------+
|laptop_ID|Company|        Product| TypeName|Inches|    ScreenResolution|                 Cpu| Ram|             Memory|                 Gpu|     OpSys|Weight|Price_euros|
+---------+-------+---------------+---------+------+--------------------+--------------------+----+-------------------+--------------------+----------+------+-----------+
|        1|  Apple|    MacBook Pro|Ultrabook|  13.3|IPS Panel Retina ...|Intel Core i5 2.3GHz| 8GB|          128GB SSD|Intel Iris Plus G...|     macOS|  1.37|    1339.69|
|        2|  Apple|    Macbook Air|Ultrabook|  13.3|            1440x900|Intel Core i5 1.8GHz| 8GB|128GB Flash Storage|Intel HD Graphics...|     macOS|  1.34|     898.94|
|        3|     HP|         250 G6| Notebook|  15.6|   Full HD 1920x1080|Intel Core i5 720...| 8GB|          256GB SSD|Intel HD Graphics...|     

Check the data type after the transformation.

In [11]:
laptop.printSchema()

root
 |-- laptop_ID: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- TypeName: string (nullable = true)
 |-- Inches: double (nullable = true)
 |-- ScreenResolution: string (nullable = true)
 |-- Cpu: string (nullable = true)
 |-- Ram: string (nullable = true)
 |-- Memory: string (nullable = true)
 |-- Gpu: string (nullable = true)
 |-- OpSys: string (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Price_euros: double (nullable = true)



Let's check if there is any missing value as a result of our data wrangling process on the `Weight` column.

In [12]:
original_row = laptop.count()
after_na = laptop.na.drop().count()

print("Number of rows with missing value:")
original_row - after_na

Number of rows with missing value:


0

### Transforming RAM

The next thing we do is removing the unit `GB` from the `Ram` column and transform the value into integer.

In [13]:
laptop = laptop.withColumn('Ram', f.regexp_replace(laptop['Ram'], 'GB', ''))
laptop = laptop.withColumn('Ram', laptop['Ram'].cast(IntegerType()))

laptop.show(5)

+---------+-------+-----------+---------+------+--------------------+--------------------+---+-------------------+--------------------+-----+------+-----------+
|laptop_ID|Company|    Product| TypeName|Inches|    ScreenResolution|                 Cpu|Ram|             Memory|                 Gpu|OpSys|Weight|Price_euros|
+---------+-------+-----------+---------+------+--------------------+--------------------+---+-------------------+--------------------+-----+------+-----------+
|        1|  Apple|MacBook Pro|Ultrabook|  13.3|IPS Panel Retina ...|Intel Core i5 2.3GHz|  8|          128GB SSD|Intel Iris Plus G...|macOS|  1.37|    1339.69|
|        2|  Apple|Macbook Air|Ultrabook|  13.3|            1440x900|Intel Core i5 1.8GHz|  8|128GB Flash Storage|Intel HD Graphics...|macOS|  1.34|     898.94|
|        3|     HP|     250 G6| Notebook|  15.6|   Full HD 1920x1080|Intel Core i5 720...|  8|          256GB SSD|Intel HD Graphics...|No OS|  1.86|      575.0|
|        4|  Apple|MacBook Pro|Ult

### Transforming Memory

Now we will separate the `Memory` column into 3 different columns: SSD, HDD, and Flash based on the type of the storage system. The first thing we do is to find the specific storage type, for example SSD, and extract the value. If a laptop does not have any SSD, the value will be empty.

Here I convert TB to GB by replacing the string `TB` to `000` to indicate thousand. I also remove the unit from the string and the remaining string will only consists of the memory size and the type of the memory (SSD, HDD, Flash Storage).

In [14]:
laptop = laptop.withColumn('Memory', f.regexp_replace(laptop['Memory'], 'TB ', '000 '))
laptop = laptop.withColumn('Memory', f.regexp_replace(laptop['Memory'], 'GB ', ' '))

laptop.show(5)

+---------+-------+-----------+---------+------+--------------------+--------------------+---+-----------------+--------------------+-----+------+-----------+
|laptop_ID|Company|    Product| TypeName|Inches|    ScreenResolution|                 Cpu|Ram|           Memory|                 Gpu|OpSys|Weight|Price_euros|
+---------+-------+-----------+---------+------+--------------------+--------------------+---+-----------------+--------------------+-----+------+-----------+
|        1|  Apple|MacBook Pro|Ultrabook|  13.3|IPS Panel Retina ...|Intel Core i5 2.3GHz|  8|          128 SSD|Intel Iris Plus G...|macOS|  1.37|    1339.69|
|        2|  Apple|Macbook Air|Ultrabook|  13.3|            1440x900|Intel Core i5 1.8GHz|  8|128 Flash Storage|Intel HD Graphics...|macOS|  1.34|     898.94|
|        3|     HP|     250 G6| Notebook|  15.6|   Full HD 1920x1080|Intel Core i5 720...|  8|          256 SSD|Intel HD Graphics...|No OS|  1.86|      575.0|
|        4|  Apple|MacBook Pro|Ultrabook|  15.

The next thing is to extract how many memory of the laptop come from `SSD`. The first thing is to extract with the pattern of `\d+.* SSD` which indicates that I only want to get numbers followed by whitespace and the word SSD. If these pattern is not found, the regex will return nothing ('') and we will replace it by `0`. We also remove the whitespace and the word SSD (` SSD`) from the string.

In [15]:
laptop = laptop.withColumn('ssd', f.regexp_extract(laptop['Memory'], '\d+.* SSD', 0))
laptop = laptop.withColumn('ssd', f.regexp_replace(laptop['ssd'], ' SSD', ''))
laptop = laptop.withColumn('ssd', f.when(laptop['ssd'] == '', '0').otherwise(laptop['ssd']))

laptop.select(['laptop_ID', 'Memory', 'ssd']).show(30)

+---------+-------------------+----------+
|laptop_ID|             Memory|       ssd|
+---------+-------------------+----------+
|        1|            128 SSD|       128|
|        2|  128 Flash Storage|         0|
|        3|            256 SSD|       256|
|        4|            512 SSD|       512|
|        5|            256 SSD|       256|
|        6|            500 HDD|         0|
|        7|  256 Flash Storage|         0|
|        8|  256 Flash Storage|         0|
|        9|            512 SSD|       512|
|       10|            256 SSD|       256|
|       11|            500 HDD|         0|
|       12|            500 HDD|         0|
|       13|            256 SSD|       256|
|       14|            256 SSD|       256|
|       15|            256 SSD|       256|
|       16|            256 SSD|       256|
|       17|            256 SSD|       256|
|       18|            512 SSD|       512|
|       19|           1000 HDD|         0|
|       20|            128 SSD|       128|
|       21|

You may have notice from the above table, on the 29th rows there is an SSD with the string `256 + 256` which means that the laptop come with 2 separate SSD. We will sum the memory to get the total memory while also converting all the string into numeric type.

In [16]:
laptop = laptop.withColumn('ssd', f.when( f.col('ssd').contains('+'), 
    f.trim(f.regexp_extract(laptop['ssd'], '\d+ ', 0)).cast(DoubleType()) + f.trim(f.regexp_extract(laptop['ssd'], ' \d+', 0)).cast(DoubleType()) ).\
        otherwise(laptop['ssd'].cast(DoubleType()))
        ) 

laptop.select(['laptop_ID', 'Memory', 'ssd']).show(30)

+---------+-------------------+-----+
|laptop_ID|             Memory|  ssd|
+---------+-------------------+-----+
|        1|            128 SSD|128.0|
|        2|  128 Flash Storage|  0.0|
|        3|            256 SSD|256.0|
|        4|            512 SSD|512.0|
|        5|            256 SSD|256.0|
|        6|            500 HDD|  0.0|
|        7|  256 Flash Storage|  0.0|
|        8|  256 Flash Storage|  0.0|
|        9|            512 SSD|512.0|
|       10|            256 SSD|256.0|
|       11|            500 HDD|  0.0|
|       12|            500 HDD|  0.0|
|       13|            256 SSD|256.0|
|       14|            256 SSD|256.0|
|       15|            256 SSD|256.0|
|       16|            256 SSD|256.0|
|       17|            256 SSD|256.0|
|       18|            512 SSD|512.0|
|       19|           1000 HDD|  0.0|
|       20|            128 SSD|128.0|
|       21|   32 Flash Storage|  0.0|
|       22|128 SSD +  1000 HDD|128.0|
|       23|            500 HDD|  0.0|
|       24| 

Check the schema or the data type of each column.

In [17]:
laptop.printSchema()

root
 |-- laptop_ID: integer (nullable = true)
 |-- Company: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- TypeName: string (nullable = true)
 |-- Inches: double (nullable = true)
 |-- ScreenResolution: string (nullable = true)
 |-- Cpu: string (nullable = true)
 |-- Ram: integer (nullable = true)
 |-- Memory: string (nullable = true)
 |-- Gpu: string (nullable = true)
 |-- OpSys: string (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Price_euros: double (nullable = true)
 |-- ssd: double (nullable = true)



We will do the same thing with the HDD and Flash Storage.

In [18]:
# HDD Storage
laptop = laptop.withColumn('hdd', f.regexp_extract(laptop['Memory'], '\d+.* HDD', 0))
laptop = laptop.withColumn('hdd', f.regexp_replace(laptop['hdd'], ' HDD', ''))
laptop = laptop.withColumn('hdd', f.when(laptop['hdd'] == '', '0').otherwise(laptop['hdd']))
laptop = laptop.withColumn('hdd', f.when( f.col('hdd').contains('+'), 
    f.rtrim(f.regexp_extract(laptop['hdd'], '\d+ ', 0)).cast(DoubleType()) + f.ltrim(f.regexp_extract(laptop['hdd'], ' \d+', 0)).cast(DoubleType()) ).\
        otherwise(laptop['hdd'].cast(DoubleType()))
        ) 

# Flash Storage
laptop = laptop.withColumn('flash', f.regexp_extract(laptop['Memory'], '\d+.* Flash', 0))
laptop = laptop.withColumn('flash', f.regexp_replace(laptop['flash'], ' Flash', ''))
laptop = laptop.withColumn('flash', f.when(laptop['flash'] == '', '0').otherwise(laptop['flash']))
laptop = laptop.withColumn('flash', f.when( f.col('flash').contains('+'), 
    f.rtrim(f.regexp_extract(laptop['flash'], '\d+ ', 0)).cast(DoubleType()) + f.ltrim(f.regexp_extract(laptop['flash'], ' \d+', 0)).cast(DoubleType()) ).\
        otherwise(laptop['flash'].cast(DoubleType()))
        ) 

laptop.select(['laptop_ID', 'Memory', 'ssd', 'hdd', 'flash']).show(10)

+---------+-----------------+-----+-----+-----+
|laptop_ID|           Memory|  ssd|  hdd|flash|
+---------+-----------------+-----+-----+-----+
|        1|          128 SSD|128.0|  0.0|  0.0|
|        2|128 Flash Storage|  0.0|  0.0|128.0|
|        3|          256 SSD|256.0|  0.0|  0.0|
|        4|          512 SSD|512.0|  0.0|  0.0|
|        5|          256 SSD|256.0|  0.0|  0.0|
|        6|          500 HDD|  0.0|500.0|  0.0|
|        7|256 Flash Storage|  0.0|  0.0|256.0|
|        8|256 Flash Storage|  0.0|  0.0|256.0|
|        9|          512 SSD|512.0|  0.0|  0.0|
|       10|          256 SSD|256.0|  0.0|  0.0|
+---------+-----------------+-----+-----+-----+
only showing top 10 rows



### Transforming CPU

The next thing we do is transforming the `Cpu` column. We will separate the processor type and the processor clock speed.

The processor clock speed is indicated by the number followed by the GigaHertz (GHz) unit. Let's check the number of CPU type and their respective frequency of data.

In [19]:
laptop.groupBy('Cpu').count().sort(f.col('count').desc()).show(truncate = False)

+------------------------------------+-----+
|Cpu                                 |count|
+------------------------------------+-----+
|Intel Core i5 7200U 2.5GHz          |190  |
|Intel Core i7 7700HQ 2.8GHz         |146  |
|Intel Core i7 7500U 2.7GHz          |134  |
|Intel Core i7 8550U 1.8GHz          |73   |
|Intel Core i5 8250U 1.6GHz          |72   |
|Intel Core i5 6200U 2.3GHz          |68   |
|Intel Core i3 6006U 2GHz            |64   |
|Intel Core i7 6500U 2.5GHz          |49   |
|Intel Core i7 6700HQ 2.6GHz         |43   |
|Intel Core i3 7100U 2.4GHz          |37   |
|Intel Core i5 7300HQ 2.5GHz         |33   |
|Intel Celeron Dual Core N3350 1.1GHz|30   |
|Intel Celeron Dual Core N3060 1.6GHz|28   |
|Intel Core i7 6600U 2.6GHz          |18   |
|Intel Core i3 6006U 2.0GHz          |16   |
|Intel Core i5 7300U 2.6GHz          |14   |
|Intel Pentium Quad Core N4200 1.1GHz|14   |
|Intel Core i7 7600U 2.8GHz          |13   |
|Intel Core i5 6300U 2.4GHz          |11   |
|Intel Cel

To simplify the processor/CPU type and prevent us from getting to many categorical class, we will only consider the general type only. For example, `Intel Core i5` and `Intel Core i5 7200U` will be considered as the same type of CPU. Let's check the result of the CPU type name cleansing process. We expect a general CPU type and try not to be to specific to reduce number of new features.

In [20]:
laptop = laptop.withColumn('cpu_type', f.regexp_extract(laptop['Cpu'], '.*? \d+', 0))
laptop = laptop.withColumn('cpu_type', f.regexp_replace(laptop['cpu_type'], ' \d+.*', ''))
laptop = laptop.withColumn('cpu_type', f.regexp_replace(laptop['cpu_type'], '[-].*', ''))
laptop = laptop.withColumn('cpu_type', f.rtrim(f.regexp_replace(laptop['cpu_type'], '[A-Z]\d+.*', '')))

laptop.groupBy('cpu_type').count().sort(f.col('count').desc()).show(truncate = False)

+-----------------------+-----+
|cpu_type               |count|
+-----------------------+-----+
|Intel Core i7          |527  |
|Intel Core i5          |423  |
|Intel Core i3          |136  |
|Intel Celeron Dual Core|80   |
|AMD                    |47   |
|Intel Pentium Quad Core|27   |
|Intel Core M           |16   |
|Intel Atom x5          |10   |
|AMD E                  |9    |
|Intel Celeron Quad Core|8    |
|AMD Ryzen              |4    |
|Intel Xeon             |4    |
|Intel Atom             |3    |
|Intel Pentium Dual Core|3    |
|Intel Core M m3        |2    |
|AMD FX                 |2    |
|Samsung Cortex         |1    |
|Intel Core M m7        |1    |
+-----------------------+-----+



We will continue by extracting the CPU clock speed.

In [21]:
# CPU Clock Speed
laptop = laptop.withColumn('cpu_clock', f.regexp_extract(laptop['Cpu'], ' \d+GHz|\d+[.]\d+.*GHz', 0))
laptop = laptop.withColumn('cpu_clock', f.regexp_replace(laptop['cpu_clock'], 'GHz', '').cast(DoubleType()))

laptop.select(['laptop_ID', 'Cpu', 'cpu_type', 'cpu_clock']).show(10)

+---------+--------------------+-------------+---------+
|laptop_ID|                 Cpu|     cpu_type|cpu_clock|
+---------+--------------------+-------------+---------+
|        1|Intel Core i5 2.3GHz|Intel Core i5|      2.3|
|        2|Intel Core i5 1.8GHz|Intel Core i5|      1.8|
|        3|Intel Core i5 720...|Intel Core i5|      2.5|
|        4|Intel Core i7 2.7GHz|Intel Core i7|      2.7|
|        5|Intel Core i5 3.1GHz|Intel Core i5|      3.1|
|        6|AMD A9-Series 942...|          AMD|      3.0|
|        7|Intel Core i7 2.2GHz|Intel Core i7|      2.2|
|        8|Intel Core i5 1.8GHz|Intel Core i5|      1.8|
|        9|Intel Core i7 855...|Intel Core i7|      1.8|
|       10|Intel Core i5 825...|Intel Core i5|      1.6|
+---------+--------------------+-------------+---------+
only showing top 10 rows



### Transforming GPU

GPU is also an important part, especially for people who want to look for better gaming experience. Since there are a lot of GPU variant, we will only extract the first 2 words from the GPU type. For example, `Intel Iris` or `Intel HD`.

In [22]:
laptop = laptop.withColumn('gpu_type', f.concat( f.split(laptop['Gpu'], ' ').getItem(0) , f.lit(' '), f.split(laptop['Gpu'], ' ').getItem(1)) )

laptop.select(['laptop_ID', 'Gpu', 'gpu_type']).show(10, truncate = False)

+---------+----------------------------+--------------+
|laptop_ID|Gpu                         |gpu_type      |
+---------+----------------------------+--------------+
|1        |Intel Iris Plus Graphics 640|Intel Iris    |
|2        |Intel HD Graphics 6000      |Intel HD      |
|3        |Intel HD Graphics 620       |Intel HD      |
|4        |AMD Radeon Pro 455          |AMD Radeon    |
|5        |Intel Iris Plus Graphics 650|Intel Iris    |
|6        |AMD Radeon R5               |AMD Radeon    |
|7        |Intel Iris Pro Graphics     |Intel Iris    |
|8        |Intel HD Graphics 6000      |Intel HD      |
|9        |Nvidia GeForce MX150        |Nvidia GeForce|
|10       |Intel UHD Graphics 620      |Intel UHD     |
+---------+----------------------------+--------------+
only showing top 10 rows



### Transforming Screen Resolution

Next, we will extract information from the `ScreenResolution`. If the laptop has touchscreen feature, we will give value of `1`.

In [23]:
laptop = laptop.withColumn('touch_screen', f.when( f.col('ScreenResolution').contains('Touchscreen'), 1).otherwise(0))

laptop.select(['laptop_ID', 'ScreenResolution', 'touch_screen']).show(10, truncate = False)

+---------+----------------------------------+------------+
|laptop_ID|ScreenResolution                  |touch_screen|
+---------+----------------------------------+------------+
|1        |IPS Panel Retina Display 2560x1600|0           |
|2        |1440x900                          |0           |
|3        |Full HD 1920x1080                 |0           |
|4        |IPS Panel Retina Display 2880x1800|0           |
|5        |IPS Panel Retina Display 2560x1600|0           |
|6        |1366x768                          |0           |
|7        |IPS Panel Retina Display 2880x1800|0           |
|8        |1440x900                          |0           |
|9        |Full HD 1920x1080                 |0           |
|10       |IPS Panel Full HD 1920x1080       |0           |
+---------+----------------------------------+------------+
only showing top 10 rows



Now we will extract the screen width. A special is when the screen resolution is in 4K, where the dimension is not explicitly stated. To counter such problem, we will assume that for all laptop with 4K resolution has aspect ratio of 16:9 or 3840x2160, which is the most common 4K resolution according to [PC Monitors](https://pcmonitors.info/articles/the-4k-uhd-3840-x-2160-experience/).

In [24]:
laptop = laptop.withColumn('screen_width', f.regexp_extract(laptop['ScreenResolution'], '\d+.*?x',0))
laptop = laptop.withColumn('screen_width', f.regexp_replace(laptop['screen_width'], 'x', ''))
laptop = laptop.withColumn('screen_width', f.when( f.col('screen_width').contains('4K'), '3840').otherwise(f.col('screen_width')).\
    cast(DoubleType()))

laptop.select(['laptop_ID', 'ScreenResolution', 'touch_screen', 'screen_width']).show(10, truncate = False)

+---------+----------------------------------+------------+------------+
|laptop_ID|ScreenResolution                  |touch_screen|screen_width|
+---------+----------------------------------+------------+------------+
|1        |IPS Panel Retina Display 2560x1600|0           |2560.0      |
|2        |1440x900                          |0           |1440.0      |
|3        |Full HD 1920x1080                 |0           |1920.0      |
|4        |IPS Panel Retina Display 2880x1800|0           |2880.0      |
|5        |IPS Panel Retina Display 2560x1600|0           |2560.0      |
|6        |1366x768                          |0           |1366.0      |
|7        |IPS Panel Retina Display 2880x1800|0           |2880.0      |
|8        |1440x900                          |0           |1440.0      |
|9        |Full HD 1920x1080                 |0           |1920.0      |
|10       |IPS Panel Full HD 1920x1080       |0           |1920.0      |
+---------+----------------------------------+-----

We will continue extracting the height resolution of the screen.

In [25]:
laptop = laptop.withColumn('screen_height', f.regexp_extract(laptop['ScreenResolution'], 'x.*\d+',0))
laptop = laptop.withColumn('screen_height', f.regexp_replace(laptop['screen_height'], 'x', ''))
laptop = laptop.withColumn('screen_height', f.when( f.col('screen_height').contains('4K'), '2160').otherwise(f.col('screen_height')).\
    cast(DoubleType()))

laptop.select(['laptop_ID', 'ScreenResolution', 'touch_screen', 'screen_width', 'screen_height']).show(10, truncate = False)

+---------+----------------------------------+------------+------------+-------------+
|laptop_ID|ScreenResolution                  |touch_screen|screen_width|screen_height|
+---------+----------------------------------+------------+------------+-------------+
|1        |IPS Panel Retina Display 2560x1600|0           |2560.0      |1600.0       |
|2        |1440x900                          |0           |1440.0      |900.0        |
|3        |Full HD 1920x1080                 |0           |1920.0      |1080.0       |
|4        |IPS Panel Retina Display 2880x1800|0           |2880.0      |1800.0       |
|5        |IPS Panel Retina Display 2560x1600|0           |2560.0      |1600.0       |
|6        |1366x768                          |0           |1366.0      |768.0        |
|7        |IPS Panel Retina Display 2880x1800|0           |2880.0      |1800.0       |
|8        |1440x900                          |0           |1440.0      |900.0        |
|9        |Full HD 1920x1080               

We will also extract the type of the monitor. If an observation doesn't have any type of monitor and only show the screen resolution, we will fill the monitor type with `others`.

In [26]:
laptop = laptop.withColumn('monitor_type', f.regexp_replace(laptop['ScreenResolution'], '\d+.*', ''))
laptop = laptop.withColumn('monitor_type', f.regexp_replace(laptop['monitor_type'], 'Touchscreen', ''))
laptop = laptop.withColumn('monitor_type', f.trim(f.regexp_replace(laptop['monitor_type'], '[/]', '')))

laptop.groupBy('monitor_type').count().sort(f.col('count').desc()).show(truncate = False)

+------------------------+-----+
|monitor_type            |count|
+------------------------+-----+
|Full HD                 |555  |
|                        |364  |
|IPS Panel Full HD       |288  |
|IPS Panel               |49   |
|Quad HD+                |19   |
|IPS Panel Retina Display|17   |
|IPS Panel Quad HD+      |11   |
+------------------------+-----+



As we can see, there is a lot of laptop with no specified monitor type. We will use `others` to fill the empty monitor type value.

In [27]:
laptop = laptop.withColumn('monitor_type', f.when(f.col('monitor_type') == '', 'others').otherwise(f.col('monitor_type')))

To simplify the dataset, we will drop some columns that are not necessary for building the model.

In [28]:
laptop_clean = laptop.drop('laptop_ID', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu')

laptop_clean.show(10)

+-------+------+---+----------+------+-----------+-----+-----+-----+-------------+---------+--------------+------------+------------+-------------+--------------------+
|Company|Inches|Ram|     OpSys|Weight|Price_euros|  ssd|  hdd|flash|     cpu_type|cpu_clock|      gpu_type|touch_screen|screen_width|screen_height|        monitor_type|
+-------+------+---+----------+------+-----------+-----+-----+-----+-------------+---------+--------------+------------+------------+-------------+--------------------+
|  Apple|  13.3|  8|     macOS|  1.37|    1339.69|128.0|  0.0|  0.0|Intel Core i5|      2.3|    Intel Iris|           0|      2560.0|       1600.0|IPS Panel Retina ...|
|  Apple|  13.3|  8|     macOS|  1.34|     898.94|  0.0|  0.0|128.0|Intel Core i5|      1.8|      Intel HD|           0|      1440.0|        900.0|              others|
|     HP|  15.6|  8|     No OS|  1.86|      575.0|256.0|  0.0|  0.0|Intel Core i5|      2.5|      Intel HD|           0|      1920.0|       1080.0|        

## One-Hot Encoding

Before we split the data into data training and data testing, now we will convert the categorical variable into dummy features by using one-hot encoding so that it can be processed by the machine learing model.

The following columns will be transformed:

- cpu_type
- gpu_type
- OpSys (OS)
- Company
- Monitor Type

First, we will convert each category into integer index. For example, `Intel Core i7` will be 0, `Intel Core i5` will be 1, `AMD` will be 4, etc.

In [29]:
cpu_str_indexer = StringIndexer(inputCol='cpu_type', outputCol = 'cpu_index').fit(laptop_clean)

laptop_clean = cpu_str_indexer.transform(laptop_clean)

laptop_clean.select(['cpu_type','cpu_index']).show(10) 

+-------------+---------+
|     cpu_type|cpu_index|
+-------------+---------+
|Intel Core i5|      1.0|
|Intel Core i5|      1.0|
|Intel Core i5|      1.0|
|Intel Core i7|      0.0|
|Intel Core i5|      1.0|
|          AMD|      4.0|
|Intel Core i7|      0.0|
|Intel Core i5|      1.0|
|Intel Core i7|      0.0|
|Intel Core i5|      1.0|
+-------------+---------+
only showing top 10 rows



After the category has been tranfomed into index, now we can continue by doing one-hot encoding. The output is an array. For example, `(18,[1],[1.0])` means that there is 18 elements in the array and on index 1 the value is 1.0 while the other is 0.

In [30]:
cpu_encoder = OneHotEncoder(inputCol = 'cpu_index', outputCol='cpu_encoded', dropLast = False).fit(laptop_clean)

laptop_clean = cpu_encoder.transform(laptop_clean)

laptop_clean.select(['cpu_type','cpu_index', 'cpu_encoded']).show(10) 

+-------------+---------+--------------+
|     cpu_type|cpu_index|   cpu_encoded|
+-------------+---------+--------------+
|Intel Core i5|      1.0|(18,[1],[1.0])|
|Intel Core i5|      1.0|(18,[1],[1.0])|
|Intel Core i5|      1.0|(18,[1],[1.0])|
|Intel Core i7|      0.0|(18,[0],[1.0])|
|Intel Core i5|      1.0|(18,[1],[1.0])|
|          AMD|      4.0|(18,[4],[1.0])|
|Intel Core i7|      0.0|(18,[0],[1.0])|
|Intel Core i5|      1.0|(18,[1],[1.0])|
|Intel Core i7|      0.0|(18,[0],[1.0])|
|Intel Core i5|      1.0|(18,[1],[1.0])|
+-------------+---------+--------------+
only showing top 10 rows



We will continue by transforming the rest of the categorical variables.

In [31]:
# GPU 
gpu_str_indexer = StringIndexer(inputCol='gpu_type', outputCol = 'gpu_index').fit(laptop_clean)
laptop_clean = gpu_str_indexer.transform(laptop_clean)

gpu_encoder = OneHotEncoder(inputCol = 'gpu_index', outputCol='gpu_encoded', dropLast = False).fit(laptop_clean)
laptop_clean = gpu_encoder.transform(laptop_clean)

# OS
os_str_indexer = StringIndexer(inputCol='OpSys', outputCol = 'os_index').fit(laptop_clean)
laptop_clean = os_str_indexer.transform(laptop_clean)

os_encoder = OneHotEncoder(inputCol = 'os_index', outputCol='os_encoded', dropLast = False).fit(laptop_clean)
laptop_clean = os_encoder.transform(laptop_clean)

# Company
comp_str_indexer = StringIndexer(inputCol='Company', outputCol = 'comp_index').fit(laptop_clean)
laptop_clean = comp_str_indexer.transform(laptop_clean)

comp_encoder = OneHotEncoder(inputCol = 'comp_index', outputCol='comp_encoded', dropLast = False).fit(laptop_clean)
laptop_clean = comp_encoder.transform(laptop_clean)

# Monitor Type
mon_str_indexer = StringIndexer(inputCol='monitor_type', outputCol = 'mon_index').fit(laptop_clean)
laptop_clean = mon_str_indexer.transform(laptop_clean)

mon_encoder = OneHotEncoder(inputCol = 'mon_index', outputCol='mon_encoded', dropLast = False).fit(laptop_clean)
laptop_clean = mon_encoder.transform(laptop_clean)

Finally, we will drop unnecessary column.

In [32]:
laptop_final = laptop_clean.drop('Company', 'OpSys', 'cpu_type', 'gpu_type', 'monitor_type', 'cpu_index', 'gpu_index', 'mon_index', 'comp_index')

laptop_final.show(5)

+------+---+------+-----------+-----+---+-----+---------+------------+------------+-------------+--------------+--------------+--------+-------------+--------------+-------------+
|Inches|Ram|Weight|Price_euros|  ssd|hdd|flash|cpu_clock|touch_screen|screen_width|screen_height|   cpu_encoded|   gpu_encoded|os_index|   os_encoded|  comp_encoded|  mon_encoded|
+------+---+------+-----------+-----+---+-----+---------+------------+------------+-------------+--------------+--------------+--------+-------------+--------------+-------------+
|  13.3|  8|  1.37|    1339.69|128.0|0.0|  0.0|      2.3|           0|      2560.0|       1600.0|(18,[1],[1.0])|(12,[5],[1.0])|     5.0|(9,[5],[1.0])|(19,[7],[1.0])|(7,[5],[1.0])|
|  13.3|  8|  1.34|     898.94|  0.0|0.0|128.0|      1.8|           0|      1440.0|        900.0|(18,[1],[1.0])|(12,[0],[1.0])|     5.0|(9,[5],[1.0])|(19,[7],[1.0])|(7,[1],[1.0])|
|  15.6|  8|  1.86|      575.0|256.0|0.0|  0.0|      2.5|           0|      1920.0|       1080.0|(18

Finally, we will create an assembler to join the feature together into a single array. First, we define which column will be the feature.

In [33]:
feature_col = laptop_final.columns
feature_col.remove('Price_euros')

In [34]:
laptop_assembler = VectorAssembler(inputCols = feature_col, outputCol = 'features')
laptop_ml = laptop_assembler.transform(laptop_final).select('Price_euros', 'features')

laptop_ml.show(10, truncate = False)

+-----------+-------------------------------------------------------------------------------------------------------+
|Price_euros|features                                                                                               |
+-----------+-------------------------------------------------------------------------------------------------------+
|1339.69    |(76,[0,1,2,3,6,8,9,11,33,40,46,57,74],[13.3,8.0,1.37,128.0,2.3,2560.0,1600.0,1.0,1.0,5.0,1.0,1.0,1.0]) |
|898.94     |(76,[0,1,2,5,6,8,9,11,28,40,46,57,70],[13.3,8.0,1.34,128.0,1.8,1440.0,900.0,1.0,1.0,5.0,1.0,1.0,1.0])  |
|575.0      |(76,[0,1,2,3,6,8,9,11,28,40,42,52,69],[15.6,8.0,1.86,256.0,2.5,1920.0,1080.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
|2537.45    |(76,[0,1,2,3,6,8,9,10,30,40,46,57,74],[15.4,16.0,1.83,512.0,2.7,2880.0,1800.0,1.0,1.0,5.0,1.0,1.0,1.0])|
|1803.6     |(76,[0,1,2,3,6,8,9,11,33,40,46,57,74],[13.3,8.0,1.37,256.0,3.1,2560.0,1600.0,1.0,1.0,5.0,1.0,1.0,1.0]) |
|400.0      |(76,[0,1,2,4,6,8,9,14,30,41,54,70],[15.6,4.

The next thing is splitting the data into training and testing dataset. We will use 20% of the data as the testing set.

In [35]:
(train_data, test_data) = laptop_ml.randomSplit([0.8,0.2], seed = 123)

print("Number of Training Set: " + str(train_data.count()))
print("Number of Testing Set: " + str(test_data.count()))

Number of Training Set: 1041
Number of Testing Set: 262


Now we will normalize all features so that they will have the same scale.

In [36]:
feature_scaler = StandardScaler(inputCol = 'features', outputCol = 'scaled_feature').fit(train_data)

train_data = feature_scaler.transform(train_data)
test_data = feature_scaler.transform(test_data)

## Model Fitting

We will start building machine learning model. We will build the following model and compare the predictive performance:

- Linear Regression
- Lasso Regression 
- Ridge Regression 
- Elastic Net Regression

### Linear Regression

First, we fit the OLS (Ordinary Least Square) linear regression into the training dataset. OLS will try to find the best coefficient for the intercept and each feature by minimizing the **Sum of Squared Error** as the lost function.

$$
SSE = \Sigma_{i=1}^n (y - \overline y)^2
$$



In [37]:
linear_model = LinearRegression(featuresCol = 'scaled_feature', labelCol = 'Price_euros').fit(train_data)

Let's check the estimate coefficients for each features.

In [38]:
pd.DataFrame({'Coefficients' : list(linear_model.coefficients)})

,Coefficients
0,-104.784682
1,231.813028
2,154.769230
3,172.472791
4,16.840297
...,...
71,9.607375
72,13.593014
73,5.798212
74,8.850902


#### Model Evaluation

Let's check the prediction performance of the linear regression. We will use the R-squared (R2 Score) and the error measured by Root Mean Squared Error (RMSE). RMSE is a good measure to evaluate regression problem because they punish model more if there are observations that has high error.

In [39]:
pred_test = linear_model.transform(test_data)

pred_test.show(10)

+-----------+--------------------+--------------------+------------------+
|Price_euros|            features|      scaled_feature|        prediction|
+-----------+--------------------+--------------------+------------------+
|      196.0|(76,[0,1,2,5,6,8,...|(76,[0,1,2,5,6,8,...|395.10240043764685|
|      209.0|(76,[0,1,2,3,6,8,...|(76,[0,1,2,3,6,8,...| 177.1926571090077|
|      229.0|(76,[0,1,2,3,6,8,...|(76,[0,1,2,3,6,8,...| 389.6019183241022|
|      229.0|(76,[0,1,2,5,6,8,...|(76,[0,1,2,5,6,8,...| 260.6918663586507|
|      245.0|(76,[0,1,2,5,6,8,...|(76,[0,1,2,5,6,8,...| 272.6768573416165|
|      249.0|(76,[0,1,2,5,6,8,...|(76,[0,1,2,5,6,8,...| 303.0795790008959|
|      265.0|(76,[0,1,2,3,6,8,...|(76,[0,1,2,3,6,8,...| 351.6796525465811|
|      265.0|(76,[0,1,2,4,6,8,...|(76,[0,1,2,4,6,8,...| 475.5192679670997|
|     270.62|(76,[0,1,2,3,6,8,...|(76,[0,1,2,3,6,8,...|121.79844227990338|
|      272.0|(76,[0,1,2,4,6,8,...|(76,[0,1,2,4,6,8,...| 162.5798409150588|
+-----------+------------

In [40]:
print("R-Squared: {:.3f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='r2').evaluate(pred_test)))
print("RMSE: {:.4f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='rmse').evaluate(pred_test)))

R-Squared: 0.760
RMSE: 320.6822


We can compare the RMSE with the standard deviation of the price variable from the testing dataset. According to [Bowles](https://www.amazon.com/Machine-Learning-Spark-Python-Techniques/dp/1119561930), if the RMSE is lower than the standard deviation, then we can conclude that the model has a good performance. A good model should, on average, have better predictions than the naive estimate of the mean for all predictions.

In [41]:
test_data.select(f.format_number(f.stddev('Price_euros'),4).alias('Price Standard Deviation')).show()

+------------------------+
|Price Standard Deviation|
+------------------------+
|                656.0568|
+------------------------+



### Lasso Regression

Lasso regression is a variant of linear regression that comes with a penalty on the loss function to help the model do regularization and reduce the model variance. Model with less variance will be better at predicing new data. The idea is to induce the penalty against complexity by adding the regularization term such as that with increasing value of regularization parameter, the weights get reduced (and, hence penalty induced).

As you may have learn before, linear regression try to get the best estimate value for the model intercept and slope for each feature by minimizing the Sum of Squared Error (SSE). 

$$
SSE = \Sigma_i^N (y_i - \hat y_i)^2
$$

Lasso Regression will add an L1 penalty with $\lambda$ constant to the loss function. If $\lambda$ equals zero, then the lasso regression become identical with the ordinary linear regression.

$$
SSE = \Sigma_{i=1}^N (y_i - \hat y_i)^2 + \lambda\ \Sigma_{j=1}^n |\beta_j|
$$

The benefit of using Lasso is that it can function as a feature selection method. This model will shrink and sometimes remove features so that we only have the features that affect the target data. To fit a Lasso model, we need to scale all features. The features need to have the same scale so that the coefficient values are chosen based only on which attribute is most useful, not on the basis of which one has the most favorable scale.

The first thing we need to do to build a Lasso model is by choosing the appropriate value of $\lambda$ as the penalty constant. pyspark has build in estimator can help us get the optimal hyper-parameter (in this case, $\lambda$) with Cross-Validation method with grid search to evaluate the model. For a Lasso model, the `elasticNetParam` must be set to 1.

In the following step, we set 10-Fold Cross-Validation method to fit and evaluate the data and try 1000 different $\lambda$ as the penalty constant. The model will give us the best \$lambda$ to choose.

In [83]:
# Set Lasso with elasticNetParam = 1
lr = LinearRegression(elasticNetParam = 1, featuresCol = 'scaled_feature', labelCol = 'Price_euros') # Lasso

# Set the value of lambda to tune
paramgrid = ParamGridBuilder().addGrid(lr.regParam, np.linspace(0, 10, 100)).build()

# Set error metric to evaluate
reg_evaluator = RegressionEvaluator(labelCol='Price_euros', metricName='rmse')

# Set Cross Validation setting
lasso_cv = CrossValidator(estimator = lr, estimatorParamMaps = paramgrid, evaluator = reg_evaluator, numFolds = 10, parallelism = 4)

# Model Fitting
lasso_cv = lasso_cv.fit(train_data)

print("Best Lambda: {:.4f}".format(lasso_cv.bestModel.getRegParam()))

Best Lambda: 2.8283


In [84]:
pred_lasso = lasso_cv.transform(test_data)

print("R-Squared: {:.3f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='r2').evaluate(pred_lasso)))
print("RMSE: {:.4f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='rmse').evaluate(pred_lasso)))

R-Squared: 0.761
RMSE: 320.2203


### Ridge Regression

Ridge regression is similar with Lasso by creating a penalty toward the lost function. The difference is that the ridge regression will square the coefficient instead of making it absolute for the penalty. Larger value of $\lambda$ will make the coefficient to be smaller, but never reach to 0 in Ridge regression.

$$
SSE = \Sigma_{i=1}^N (y_i - \hat y_i)^2 + \lambda\ \Sigma_{j=1}^n \beta_j^2
$$

In the following process, I set the possible alpha values from 0.0001 to 100 with different steps.

In [85]:
# Set Ridge with elasticNetParam = 0
lr = LinearRegression(elasticNetParam = 0, featuresCol = 'scaled_feature', labelCol = 'Price_euros') # Lasso

# Set the value of lambda to tune
lambda_range = [1e-4, 1e-3, 1e-2, 0.1, 1]
lambda_range.extend(np.arange(10, 100, 1))
paramgrid = ParamGridBuilder().addGrid(lr.regParam, lambda_range).build()

# Set error metric to evaluate
reg_evaluator = RegressionEvaluator(labelCol='Price_euros', metricName='rmse')

# Set Cross Validation setting
ridge_cv = CrossValidator(estimator = lr, estimatorParamMaps = paramgrid, evaluator = reg_evaluator, numFolds = 10, parallelism = 4)

# Model Fitting
ridge_cv = ridge_cv.fit(train_data)

print("Best Lambda: {:.4f}".format(ridge_cv.bestModel.getRegParam()))

Best Lambda: 64.0000


Let's evaluate the model.

In [86]:
pred_ridge = ridge_cv.transform(test_data)

print("R-Squared: {:.3f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='r2').evaluate(pred_ridge)))
print("RMSE: {:.4f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='rmse').evaluate(pred_ridge)))

R-Squared: 0.768
RMSE: 315.3129


### Elastic Net Regression

Elastic Net combine both L1 penalty and the L2 penalty into a single formula. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge.

$$
SSE = \Sigma_{i=1}^N (y_i - \hat y_i)^2 + + \lambda\ \alpha\ \Sigma_{j=1}^n |\beta_j| + \lambda\ (1-\alpha)\ \frac{1}{2} \Sigma_{j=1}^n \beta_j^2
$$

Where $\lambda$ is set by `regParam` and $\alpha$ is set by `elasticNetParam` in pyspark. First, we will try to balance the L1 and the L2 penalty by using the `elasticNetParam` = 0.5

In [87]:
# Set Elastic Net alpha to 0.5
lr = LinearRegression(elasticNetParam = 0.5, featuresCol = 'scaled_feature', labelCol = 'Price_euros') # Lasso

# Set the value of lambda to tune
lambda_range = [1e-4, 1e-3, 1e-2, 0.1, 1]
lambda_range.extend(np.arange(10, 100, 1))
paramgrid = ParamGridBuilder().addGrid(lr.regParam, lambda_range).build()

# Set error metric to evaluate
reg_evaluator = RegressionEvaluator(labelCol='Price_euros', metricName='rmse')

# Set Cross Validation setting
elastic_cv = CrossValidator(estimator = lr, estimatorParamMaps = paramgrid, evaluator = reg_evaluator, numFolds = 10, parallelism = 4)

# Model Fitting
elastic_cv = elastic_cv.fit(train_data)

print("Best Lambda: {:.4f}".format(elastic_cv.bestModel.getRegParam()))
print("Alpha: 0.5")

Best Lambda: 10.0000
Alpha: 0.5


Let's evaluate the model.

In [88]:
pred_elastic = elastic_cv.transform(test_data)

print("R-Squared: {:.3f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='r2').evaluate(pred_elastic)))
print("RMSE: {:.4f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='rmse').evaluate(pred_elastic)))

R-Squared: 0.760
RMSE: 320.9835


We can put multiple `elasticNetParam` to find the best ratio between the L1 and the L2 penalty.

In [89]:
# Set Basic Linear Model
lr = LinearRegression(featuresCol = 'scaled_feature', labelCol = 'Price_euros') # Lasso

# Set the value of lambda and alpha to tune
lambda_range = [1e-4, 1e-3, 1e-2, 0.1, 1]
lambda_range.extend(np.arange(10, 100, 1))

alpha_range = [0.05, 0.1, 0.2, 0.3, 0.7, 0.8, 0.9, 0.95]

paramgrid = ParamGridBuilder().addGrid(lr.regParam, lambda_range).addGrid(lr.elasticNetParam, alpha_range).build()

# Set error metric to evaluate
reg_evaluator = RegressionEvaluator(labelCol='Price_euros', metricName='rmse')

# Set Cross Validation setting
elastic_cv = CrossValidator(estimator = lr, estimatorParamMaps = paramgrid, evaluator = reg_evaluator, numFolds = 10, parallelism = 4)

# Model Fitting
elastic_cv = elastic_cv.fit(train_data)

print("Best Lambda: {:.4f}".format(elastic_cv.bestModel.getRegParam()))
print("Best Alpha: {:.4f}".format(elastic_cv.bestModel.getElasticNetParam()))

Best Lambda: 35.0000
Best Alpha: 0.0500


Let's evaluate the model.

In [90]:
pred_elastic = elastic_cv.transform(test_data)
 
print("R-Squared: {:.3f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='r2').evaluate(pred_elastic)))
print("RMSE: {:.4f}".format(RegressionEvaluator(labelCol='Price_euros', metricName='rmse').evaluate(pred_elastic)))

R-Squared: 0.765
RMSE: 317.3814


# Conclusion

Based on our result, all regularization method works better than the vanilla linear regression, with the Ridge Regression achieve the lowest error on testing dataset. We also see that even with linear model we can achieve good result, as the RMSE of each model is still better than the standard deviation of the testing dataset. 